In [11]:
# coding: utf-8

# In[1]:




# In[1]:
get_ipython().magic(u'matplotlib inline')

# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()


# In[2]:

'3.5.2'

In [12]:
# In[2]:


try:
    import scrapy
except:
    get_ipython().system('python3 -m pip install scrapy')
    import scrapy
from scrapy.crawler import CrawlerProcess
class DmozItem(scrapy.Item):
    date=scrapy.Field()
    title = scrapy.Field()
    studio=scrapy.Field()


# In[3]:

In [13]:
# In[3]:


import json

class JsonWriterPipeline2(object):

    def open_spider(self, spider):
        self.file = open('quoterest2.jl', 'w+')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item


# In[4]:

In [14]:
# In[4]:


class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoterest.jl', 'w+')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item


# In[5]:

In [15]:
# In[5]:


class JsonWriterPipeline3(object):

    def open_spider(self, spider):
        self.file = open('quoterest3.jl', 'w+')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [16]:
# In[6]:


import logging

class QuotesSpiderVan(scrapy.Spider):
    name = "quotesVan"
    start_urls = [
        'http://vfxvancouver.com/page/1/?s']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline3': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoterest3.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for sel in response.xpath(".//article[contains(@id, 'post')]"):
            print( sel)
            item = DmozItem()
            item["date"] = sel.xpath('''.//div[@class='entry-meta']/data//text()''').extract()       
            item["title"]= sel.xpath(".//h1[@class='entry-title']/a/text()").extract()
            item["studio"]= sel.xpath(".//a[@class='author-link']/@href").extract()

            print( item)
            if len(item["date"])==0:
                item["date"].append("jan 1, 2018")
            if len(item["title"])==0:
                item["title"].append("none")
            if len(item["studio"])==0:
                item["studio"].append("none")      
            dat=item["date"][0]
            ti=item["title"][0]
            st=item["studio"][0]

            
            yield {'date':dat,'title':ti,'studio':st,'city':"Vancouver"}
        url_older=response.xpath('.//a[@class="next page-numbers"]//@href').extract()
        if (len(url_older))>0:
            print (url_older)
            abs_url=url_older[0]
            print (abs_url )
            yield scrapy.Request(abs_url) 
        else:
            print("Last element")
        return

In [17]:
# In[7]:




class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://www.torontovfxjobs.com/jobs/']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoterest.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for sel in response.xpath("//header[@class='entry-header']"):
            print( sel)
            item = DmozItem()
            item["date"] = sel.xpath('''.//div[@class='meta-below-title']//time[@class="dt-published published entry-date"]//text()''').extract()       
            item["title"]= sel.xpath(".//a[@class='entry-title-passthrough']/text()").extract()
            item["studio"]= sel.xpath(".//a[@class='entry-title-passthrough']/@href").extract()

            print( item)
            if len(item["date"])==0:
                item["date"].append("jan 1, 2018")
            if len(item["title"])==0:
                item["title"].append("none")
            if len(item["studio"])==0:
                item["studio"].append("none")      
            dat=item["date"][0]
            ti=item["title"][0]
            st=item["studio"][0]

            
            yield {'date':dat,'title':ti,'studio':st,'city':"Toronto"}
        url_older=response.xpath('//div[@class="older"]//@href').extract()
        if (len(url_older))>0:
            print (url_older)
            abs_url=response.urljoin(url_older[0].split("/")[-1])
            print (abs_url )
            yield scrapy.Request(abs_url) 
        else:
            print("Last element")
        return

In [18]:
# In[8]:


# In[6]:


class JobSpider(scrapy.Spider):
    name = 'job'
    allowed_domains = ['vfx-montreal.com/jobs/']
    start_urls = ['http://vfx-montreal.com/jobs/']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline2': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoterest2.json'  }                      # Used for pipe
    def parse(self, response):
        blocks=response.xpath('//div[@class="mk-text-block  "]')
        i=0
        for block in blocks:
            titleJob=block.xpath ('.//div[@class="vfx-job-list-item-title"]/a/text()').extract_first()
            dateJob=block.xpath ('.//div[@class="vfx-job-list-item-end-date"]/text()').extract_first()
            studioJob=block.xpath ('.//div[@class="vfx-job-list-item-studio-name"]/a/@href').extract_first()
            i=i+1
            dateJobForm=dateJob.replace("Published: ",'')
            try:
                print (titleJob)
                print (dateJob)
                print (studioJob)
                yield {'date':dateJobForm,'title':titleJob,'studio':studioJob,'city':"Montreal"}

            except:
                print('i')

In [19]:
# In[15]:


import pandas as pd
import matplotlib as mp
import os
print("1:Read local data"+"\n"+"2:Read online data")
nb= input("Choose a number")
if nb=="2":
    file='quoterest.json'
    raw = open(file, "w+")
    raw.seek(0)                       
    raw.truncate()
    raw.close()
    file= 'quoterest2.json'
    raw = open(file, "w+")
    raw.seek(0)                       
    raw.truncate()
    raw.close()
    file= 'quoterest3.json'
    raw = open(file, "w+")
    raw.seek(0)                       
    raw.truncate()
    raw.close()


process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.crawl(JobSpider)
process.crawl(QuotesSpiderVan)
process.start()

1:Read local data
2:Read online data
Choose a number2


0

0

0

0

0

0

2018-07-17 15:17:48 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-07-17 15:17:48 [scrapy.utils.log] INFO: Versions: lxml 4.2.3.0, libxml2 2.9.5, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 18.7.0, Python 3.5.2 (v3.5.2:4def2a2901a5, Jun 25 2016, 22:18:55) [MSC v.1900 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.1.0h  27 Mar 2018), cryptography 2.2.2, Platform Windows-10-10.0.16299-SP0
2018-07-17 15:17:48 [scrapy.crawler] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)', 'FEED_URI': 'quoterest.json', 'FEED_FORMAT': 'json', 'LOG_LEVEL': 30}


ReactorNotRestartable: 

In [ ]:
# In[13]:


dfjson = pd.read_json('quoterest.json')

print(dfjson)


dfjson2= pd.read_json('quoterest2.json')

print(dfjson2)
dfjson3= pd.read_json('quoterest3.json')

print(dfjson3)

In [ ]:
# In[16]:



if nb=="1":
    fLoc=open('quoterest.json','r')
    fLoc2=open('quoterest2.json',"r")
    fLoc3=open('quoterest3.json',"r")
    fSize=os.stat('quoterest.json').st_size
    fSize2=os.stat('quoterest2.json').st_size
    fSize3=os.stat('quoterest3.json').st_size
    if (fSize!=0)&(fSize2!=0)&(fSize3!=0):
        dfjson = pd.read_json(fLoc)
        dfjsonConv=pd.to_datetime(dfjson['date'],infer_datetime_format=True)
        print(dfjsonConv)


        dfjson2= pd.read_json(fLoc2)

        print(dfjson2)
        dfjson3= pd.read_json(fLoc3)

        print(dfjson3)

        frames=[dfjson,dfjson2,dfjson3]
        dfConcat=pd.concat(frames)
        dfConcat.index=dfConcat['date']
        dfConcatRes=dfConcat.sort_index()

        print(dfConcatRes)
        dfNewComp=dfConcatRes[dfConcatRes["title"].str.contains("Compositor|compsitor|Comper|COMPOSITOR")]
        print(dfNewComp)
        dfNewComp.to_pickle("comper.pkl")
        dfNewGen=dfConcatRes[dfConcatRes['title'].str.contains("Generalist|GENERALIST")]
        print(dfNewGen)
        dfNewSup=dfConcatRes[dfConcatRes['title'].str.contains("Supervisor|SUPERVISOR")]
        print(dfNewSup)
        dfNewMan=dfConcatRes[dfConcatRes['title'].str.contains("Manager|Producer")]
        print(dfNewMan)
        dfNewTra=dfConcatRes[dfConcatRes['title'].str.contains("Tracking|Camera|Tracker|Matcmove|Matcmover")]
        print(dfNewTra)
        dfNewPai=dfConcatRes[dfConcatRes['title'].str.contains("PAINTER|Painter|painter|Matte|MATTE")]
        print(dfNewPai)
        dfNewAni=dfConcatRes[dfConcatRes['title'].str.contains("Animator|Animation|Rigger|RIGGER|Rig")]
        print(dfNewAni)
        dfNewMod=dfConcatRes[dfConcatRes['title'].str.contains("Modeller|MODELLER|Modeler")]
        print(dfNewMod)
        dfNewComp.groupby(dfNewComp["date"].dt.month).count().plot(kind="bar")

else:
    print("fileEmpty")

In [ ]:
# In[18]:


dfNewComp.groupby(dfNewComp["date"].dt.year).count().plot(kind="bar")


# In[ ]:


dfConcatRes.groupby(dfConcatRes["date"].dt.year).count().plot(kind="bar")